# Push Metrics Values to OpenPages Identifying Model Object by Name
This notebook will save new metric values within OpenPages, leveraging [OpenPages v2 API](https://cloud.ibm.com/apidocs/openpages).
More API usage exemples are available [here](https://github.com/IBM/watson-openscale-samples/tree/main/IBM%20Cloud/WML/notebooks/watsonx).

Connection information (URL and APIkey) is assusmed to be available in your .env file

## Notebook inputs

In [1]:
import pandas as pd
import random

MODEL_OBJECT_NAME = "INST AskHR Chatbot"

# Create a dataframe with metric values (random values between 0.90 and 0.98)
metrics_data = {
    'Metric Name': ['Answer_relevance', 'Faithfulness'],
    'Metric Value': [round(random.uniform(0.90, 0.98), 2), round(random.uniform(0.90, 0.98), 2)]
}

df_metrics = pd.DataFrame(metrics_data)
print(df_metrics)

        Metric Name  Metric Value
0  Answer_relevance          0.91
1      Faithfulness          0.97


## Prerequisites
- WX_API_KEY: IBM Cloud API Key (stored in .env file)
- OP_URL: OpenPages API Base URL (stored in .env file)
- Python 3.11+
- Required packages: requests, python-dotenv

In [2]:
# Import Required Libraries
import requests
import json
import sys
import os
from pathlib import Path
from dotenv import load_dotenv

print("Required libraries imported successfully")

Required libraries imported successfully


## Load Environment Variables

In [3]:
load_dotenv()

True

In [4]:
# Load environment variables
raw_api_key = os.getenv('WX_API_KEY', '')
raw_op_url = os.getenv('OP_URL', '')

# Parse API key: remove quotes, comments, and whitespace
WX_API_KEY = raw_api_key.split('#')[0].strip().strip('"').strip("'")
OP_URL = raw_op_url.split('#')[0].strip().strip('"').strip("'")

# Validate credentials
if not WX_API_KEY:
    raise ValueError("❌ ERROR: WX_API_KEY environment variable is not set")
if not OP_URL:
    raise ValueError("❌ ERROR: OP_URL environment variable is not set")

print(f"✓ WX_API_KEY loaded: {WX_API_KEY[:20]}...")
print(f"✓ OP_URL loaded: {OP_URL}")

✓ WX_API_KEY loaded: vzxjDTjRct-wt_F06NCj...
✓ OP_URL loaded: https://73dd332c-888e-4630-85d5-322d23f1dafd.eu-de.openpages.cloud.ibm.com


## Configuration

In [5]:
# Configuration
IAM_URL = "https://iam.cloud.ibm.com/identity/token"
OPENPAGES_API_ENDPOINT = f"{OP_URL}/opgrc/api/v2"

print(f"IAM URL: {IAM_URL}")
print(f"OpenPages API Endpoint: {OPENPAGES_API_ENDPOINT}")
print(f"Target Model Object: {MODEL_OBJECT_NAME}")

IAM URL: https://iam.cloud.ibm.com/identity/token
OpenPages API Endpoint: https://73dd332c-888e-4630-85d5-322d23f1dafd.eu-de.openpages.cloud.ibm.com/opgrc/api/v2
Target Model Object: INST AskHR Chatbot


## Authentication Helper Functions

In [6]:
def get_iam_token(api_key):
    """
    Request an IAM access token from IBM Cloud IAM service.
    
    Args:
        api_key (str): IBM Cloud API key
    
    Returns:
        str: Access token
    
    Raises:
        Exception: If token request fails
    """
    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Accept": "application/json"
    }
    
    data = {
        "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
        "apikey": api_key,
        "response_type": "cloud_iam"
    }
    
    try:
        print("🔐 Requesting IAM token...")
        response = requests.post(IAM_URL, headers=headers, data=data, timeout=10)
        response.raise_for_status()
        
        token_response = response.json()
        access_token = token_response.get('access_token')
        
        if not access_token:
            raise ValueError("No access token in response")
        
        expires_in = token_response.get('expires_in', 'unknown')
        print(f"✓ IAM token obtained successfully (expires in {expires_in}s)")
        
        return access_token
    
    except requests.exceptions.RequestException as e:
        raise Exception(f"❌ Failed to obtain IAM token: {str(e)}")
    except (KeyError, ValueError) as e:
        raise Exception(f"❌ Invalid IAM token response: {str(e)}")

## Fetch Model ID by Name

In [7]:
def fetch_model_id_by_name(headers, model_name):
    """
    Fetch a Model ID from OpenPages by name
    Uses OpenPages Query Service to search for the model object
    """
    try:        
        # Use OpenPages Query Service to find the model object by name
        get_id_payload = {
            "statement": "SELECT [Model].[Resource ID] FROM [Model] WHERE [Model].[Name] IN ('{0}')".format(model_name),
            "skipCount": 0
        }
        raw_response = requests.post(f"{OPENPAGES_API_ENDPOINT}/query", json=get_id_payload, headers=headers, verify=False)
        raw_response.raise_for_status()

        response = raw_response.json()
        model_id = None
        if response is not None:
            if response.get("rows") is not None:
                rows = response.get("rows")
                if len(rows) != 0:
                    fields = rows[0].get("fields")
                    if fields is not None:
                        if len(fields) != 0:
                            model_id = fields[0]["value"]

        if model_id is None:
            print("Model ID not found.")
        else:
            print("Model ID fetched: " + model_id)
        return model_id

    except requests.exceptions.RequestException as e:
        print(f"✗ Error fetching model id: {e}")
        return None

print("✓ Model object fetch function defined")

✓ Model object fetch function defined


## For a given model id, get the corresponding OP metrics definitions - Map containing metric id and its name

In [8]:
def get_op_model_metrics_definitions(headers, model_id):
    """
    Fetch Metrics Definition from OpenPages by model_id
    Uses OpenPages Query Service to search for metrics associated with the model
    """
    try:
        get_metrics_payload = {
            "statement": "SELECT [Metric].[Resource ID], [Metric].[Name], [Metric].[Description] FROM [Model] JOIN [Metric] ON PARENT([Model]) WHERE [Model].[Resource ID]='{0}'".format(model_id),
            "skipCount": 0
        }
        print("Sending request to fetch all metrics associated with the model.")
        response = requests.post(f"{OPENPAGES_API_ENDPOINT}/query", json=get_metrics_payload, headers=headers, verify=False).json()

        metrics_map = []

        if response is not None:
            if response.get("rows") is not None:
                rows = response.get("rows")
                if len(rows) != 0:
                    for i in range(len(rows)):
                        fields = rows[i].get("fields")
                        if fields is not None:
                            if len(fields) != 0:
                                metric_id_desc = {}
                                metric_id = None
                                metric_desc = None
                                for field in fields:
                                    if field.get('name') == 'Resource ID':
                                        metric_id = field.get('value')
                                    if field.get('name') == 'Description':
                                        metric_desc = field.get('value')
                                    if field.get('name') == 'Name':
                                        metric_id_desc['metric_name'] = field.get('value')
                                metric_id_desc['metric_desc'] = metric_desc
                                metric_id_desc['metric_id'] = metric_id
                                metrics_map.append(metric_id_desc)
            print("Completed fetching, if any, all metrics associated with the model.")
            return metrics_map
        
    except requests.exceptions.RequestException as e:
        print(f"✗ Error getting metric definitions: {e}")
        return None


## Get MetricValue Template and Build Payload

In [9]:
from datetime import datetime
import base64
import json as json_module

def get_metric_value_type_definition(headers):
    """
    Fetch the MetricValue type definition with field definitions and enum values
    """
    try:
        response = requests.get(f"{OPENPAGES_API_ENDPOINT}/types/MetricValue?includeFieldDefinitions", 
                                headers=headers, verify=False)
        response.raise_for_status()
        type_definition = response.json()
        print("✓ MetricValue type definition retrieved successfully")
        return type_definition
    except requests.exceptions.RequestException as e:
        print(f"✗ Error fetching type definition: {e}")
        raise

def get_enum_value_by_name(type_definition, field_name, enum_name):
    """
    Find enum value by field name and enum name from type definition
    
    Args:
        type_definition: Type definition response from OpenPages
        field_name: Name of the field (e.g., 'MRG-Metric-Shared:Breach Status')
        enum_name: Name of the enum value (e.g., 'Green')
    
    Returns:
        dict with enum value or None if not found
    """
    try:
        if 'field_definitions' in type_definition:
            for field_def in type_definition['field_definitions']:
                if field_def.get('name') == field_name:
                    if 'enum_values' in field_def:
                        for enum_val in field_def['enum_values']:
                            if enum_val.get('name') == enum_name:
                                return enum_val
                    break
        return None
    except Exception as e:
        print(f"⚠ Error getting enum value for {field_name}={enum_name}: {e}")
        return None

def get_metric_value_template(headers):
    """
    Fetch the MetricValue template from OpenPages
    """
    try:
        response = requests.get(f"{OPENPAGES_API_ENDPOINT}/contents/template?type_id=77", headers=headers, verify=False)
        response.raise_for_status()
        template = response.json()
        print("✓ MetricValue template retrieved successfully")
        return template
    except requests.exceptions.RequestException as e:
        print(f"✗ Error fetching template: {e}")
        raise

def extract_user_from_token(token):
    """
    Extract user information from JWT token
    
    Args:
        token: JWT access token
    
    Returns:
        dict with user information (email, name, etc.)
    """
    try:
        # JWT tokens have 3 parts separated by dots: header.payload.signature
        parts = token.split('.')
        if len(parts) != 3:
            return {"email": "Unknown", "name": "Unknown"}
        
        # Decode the payload (second part)
        payload = parts[1]
        # Add padding if needed
        padding = 4 - len(payload) % 4
        if padding != 4:
            payload += '=' * padding
        
        decoded = base64.urlsafe_b64decode(payload)
        token_data = json_module.loads(decoded)
        
        email = token_data.get('email', 'Unknown')
        name = token_data.get('name', 'Unknown')
        
        return {"email": email, "name": name}
    except Exception as e:
        print(f"⚠ Could not extract user from token: {e}")
        return {"email": "Unknown", "name": "Unknown"}

def get_metric_value_payload(template, type_definition, primaryParentId, metric_name, metric_value, token=None):
    """
    Update the MetricValue template with actual metric data
    
    Args:
        template: The template JSON object from OpenPages
        type_definition: The MetricValue type definition with enum values
        primaryParentId: Resource ID of the Metric object
        metric_name: Name of the metric
        metric_value: Value of the metric
        token: JWT access token (optional, to extract user info)
    
    Returns:
        Updated payload ready for creation
    """
    payload = template.copy()
    
    # Set the primary parent to the metric resource id
    payload['primary_parent_id'] = str(primaryParentId)
    
    # Get current date in ISO format (YYYY-MM-DD)
    current_date = datetime.now().strftime('%Y-%m-%d')
    
    # Extract user information from token
    user_info = extract_user_from_token(token) if token else {"email": "Unknown", "name": "Unknown"}
    metric_capturer = user_info.get('email', 'Unknown')
    
    # Find and update the fields
    if 'fields' in payload:
        for field in payload['fields']:
            field_name = field.get('name', '')
            
            # Set the metric value
            if field_name == 'MRG-MetricVal:Value':
                field['value'] = metric_value
            
            # Set Collection Status to "Collected" - get from type definition
            elif field_name == 'MRG-Metric-Shared:Collection Status':
                enum_val = get_enum_value_by_name(type_definition, field_name, 'Collected')
                if enum_val:
                    field['value'] = enum_val
            
            # Set the Value Date (collection date) to current date
            elif field_name == 'MRG-MetricVal:Value Date':
                field['value'] = current_date

            elif field_name == 'MRG-Metric-Shared:OpenScale Description':
                field['value'] = "Metric computed by Evidently.ai"
            
            # Set Metric Capturer to the current user
            elif field_name == 'MRG-Metric-Shared:Metric Capturer':
                field['value'] = metric_capturer
            
            # Set Metric Owner to the current user
            elif field_name == 'MRG-MetricVal:Metric Owner':
                field['value'] = metric_capturer
            
            # Set Yellow Threshold to 0.8
            elif field_name == 'MRG-Metric-Shared:Yellow Threshold':
                field['value'] = 0.8
            
            # Set Red Threshold to 0.7
            elif field_name == 'MRG-Metric-Shared:Red Threshold':
                field['value'] = 0.7
            
            # Set Breach Status to "Green" - get from type definition
            elif field_name == 'MRG-Metric-Shared:Breach Status':
                enum_val = get_enum_value_by_name(type_definition, field_name, 'Green')
                if enum_val:
                    field['value'] = enum_val
    
    print(f"✓ Payload updated for metric: {metric_name} (Value Date: {current_date}, Capturer: {metric_capturer}, Owner: {metric_capturer}, Yellow Threshold: 0.8, Red Threshold: 0.7, Breach Status: Green)")
    return payload

## Add Metric Value to the Metric Object

In [10]:
def add_metric_value_to_metric_object(headers, metric_value_payload):
    try:
        response = requests.post(f"{OPENPAGES_API_ENDPOINT}/contents", json=metric_value_payload, headers=headers, verify=False)
        response.raise_for_status()
        response_data = response.json()
        print(json.dumps(response_data, indent=2))
        metric_value_id = response_data.get('id')
        if not metric_value_id:
            raise ValueError("No 'id' in response")
        return metric_value_id
    except requests.exceptions.RequestException as e:
        print(f"✗ Error creating metric value: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        raise
    except (KeyError, ValueError) as e:
        print(f"✗ Error parsing response: {e}")
        raise

## Check for the metric existence in the metrics map

In [11]:
def get_existing_metric_id(metrics_map, metric_name):
    for item in metrics_map:
        if 'metric_name' in item and metric_name in item['metric_name']:
            return item['metric_id']
    return None

## Execute the Workflow

In [12]:
# Suppress SSL warnings
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [13]:
# Optional: Inspect Breach Status field options in template
def inspect_breach_status_options(template):
    """
    Print available options for Breach Status field to help with debugging
    """
    if 'fields' in template:
        for field in template['fields']:
            if field.get('name') == 'MRG-Metric-Shared:Breach Status':
                print("Available Breach Status options:")
                if 'options' in field:
                    print(json.dumps(field['options'], indent=2))
                else:
                    print("No options found in template field")
                break


In [14]:
# Suppress SSL warnings
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Step 1: Get IAM token
print("Step 1: Getting IAM token...")
token = get_iam_token(WX_API_KEY)

if not token:
    print("✗ Failed to obtain authentication token")
    sys.exit(1)

headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json"
}

# Step 2: Fetch the model object
print(f"\nStep 2: Fetching model object '{MODEL_OBJECT_NAME}'...")
model_id = fetch_model_id_by_name(headers, MODEL_OBJECT_NAME)

if not model_id:
    print("✗ Failed to fetch model object")
    sys.exit(1)

print("✓ Model id fetched successfully")

# Step 3: Get metrics definitions for the model
print("\nStep 3: Fetching metrics definitions...")
metrics_map = get_op_model_metrics_definitions(headers, model_id)
print("✓ Model metrics definitions fetched successfully")
print(json.dumps(metrics_map, indent=2))

# Step 4: Fetch the MetricValue template
print("\nStep 4: Fetching MetricValue template...")
template = get_metric_value_template(headers)

# Step 4b: Fetch the MetricValue type definition with enum values
print("\nStep 4b: Fetching MetricValue type definition...")
type_definition = get_metric_value_type_definition(headers)

# Step 5: Create metric values for each metric in the dataframe
print("\nStep 5: Creating metric value objects...")
for index, row in df_metrics.iterrows():
    metric_name = row['Metric Name']
    metric_value = row['Metric Value']
    
    # Check if the metric exists by the given name, and if so, get its metric_id
    metric_id = get_existing_metric_id(metrics_map, metric_name)

    # If the metric does not exist, then raise an error
    if metric_id is None:
        print(f"✗ {metric_name}: Metric Object does not exist in the model")
        raise ValueError(f"Cannot add metric value to non-existing metric object: {metric_name}")  
    
    # Build the metric value payload using the template
    # Pass token and type_definition to extract current user and dynamic enum values
    metric_value_payload = get_metric_value_payload(template, type_definition, metric_id, metric_name, metric_value, token)

    # Create the metric value - add the metric value to the metric object
    metric_value_id = add_metric_value_to_metric_object(headers, metric_value_payload)
    
    print(f"✓ {metric_name}: Metric ID: {metric_id}, Metric Value Object ID: {metric_value_id}\n")

Step 1: Getting IAM token...
🔐 Requesting IAM token...
✓ IAM token obtained successfully (expires in 3600s)

Step 2: Fetching model object 'INST AskHR Chatbot'...
Model ID fetched: 10153
✓ Model id fetched successfully

Step 3: Fetching metrics definitions...
Sending request to fetch all metrics associated with the model.
Completed fetching, if any, all metrics associated with the model.
✓ Model metrics definitions fetched successfully
[
  {
    "metric_name": "Total_records",
    "metric_desc": "Total records metric of model_health in develop phase",
    "metric_id": "10158"
  },
  {
    "metric_name": "Median_input_token_count",
    "metric_desc": "Median input token count metric of model_health in develop phase",
    "metric_id": "10161"
  },
  {
    "metric_name": "Average_output_token_count",
    "metric_desc": "Average output token count metric of model_health in develop phase",
    "metric_id": "10163"
  },
  {
    "metric_name": "Minimum_output_token_count",
    "metric_desc": 

## Summary Statistics

In [15]:
print("\n" + "="*80)
print("SUMMARY - METRIC VALUES CREATED")
print("="*80)
print(f"Model Object: {MODEL_OBJECT_NAME}")
print(f"Model Resource ID: {model_id}")
print(f"\nMetric Values Created:")
print(f"  Total metrics: {len(df_metrics)}")
for index, row in df_metrics.iterrows():
    metric_name = row['Metric Name']
    metric_value = row['Metric Value']
    metric_id = get_existing_metric_id(metrics_map, metric_name)
    print(f"    • {metric_name}: Value={metric_value}, Metric ID={metric_id}")

print(f"\nCollection Date: {datetime.now().strftime('%Y-%m-%d')}")
print("\n✓ Metric value objects successfully created and associated with model")



SUMMARY - METRIC VALUES CREATED
Model Object: INST AskHR Chatbot
Model Resource ID: 10153

Metric Values Created:
  Total metrics: 2
    • Answer_relevance: Value=0.91, Metric ID=10205
    • Faithfulness: Value=0.97, Metric ID=10197

Collection Date: 2026-01-30

✓ Metric value objects successfully created and associated with model
